In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
from datetime import datetime, date, timedelta

In [4]:
df = pd.read_csv("Nat_Gas.csv")

In [5]:
df['Dates'] = pd.to_datetime(df['Dates'])

C:\Users\shrey\AppData\Local\Temp\ipykernel_4492\2452315616.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dates'] = pd.to_datetime(df['Dates'])


In [3]:
# Function from the previous exercise
def interpolate(date, start_date, days_from_start, prices, amplitude, shift, slope, intercept):
    days = (date - pd.Timestamp(start_date)).days
    if days in days_from_start:
        # Exact match found in the data
        return prices[days_from_start.index(days)]
    else:
        # Interpolate/extrapolate using the sin/cos model
        return amplitude * np.sin(days * 2 * np.pi / 365 + shift) + days * slope + intercept

In [6]:
def price_contract(
    injection_dates,
    withdrawal_dates,
    price_df,
    injection_rate,
    withdrawal_rate,
    max_volume,
    storage_costs,
    start_date
):
    """
    Calculate the value of a gas storage contract.

    Parameters:
    - injection_dates (list of str): Dates for gas injection.
    - withdrawal_dates (list of str): Dates for gas withdrawal.
    - price_df (pd.DataFrame): DataFrame with columns 'date' and 'price'.
    - injection_rate (float): Maximum rate of gas injection (units per day).
    - withdrawal_rate (float): Maximum rate of gas withdrawal (units per day).
    - max_volume (float): Maximum storage capacity.
    - storage_costs (float): Daily storage cost per unit of gas.
    - start_date (str): Start date for interpolation.

    Returns:
    - float: Net value of the contract.
    """
    # Constants for interpolation (Taken from the previous exercise)
    amplitude = 0.8923322122175038
    shift = -0.10918070215683652
    slope = 0.0012823735987602218
    intercept = -13.517507108630003

    # Extract price data and preprocess
    #df['date'] = pd.to_datetime(price_df['date'])
    prices = df['Prices'].tolist()
    days_from_start = [(d - pd.Timestamp(start_date)).days for d in df['Dates']]

    def get_price(date):
        if date in df['Dates'].values:
            return price_df.loc[df['Dates'] == date, 'price'].iloc[0]
        else:
            return interpolate(date, start_date, days_from_start, prices, amplitude, shift, slope, intercept)

    # Validate inputs
    all_dates = sorted(set(injection_dates + withdrawal_dates))
    all_dates = [pd.Timestamp(d) for d in all_dates]

    # Initialize variables
    storage_volume = 0  # Current volume in storage
    net_value = 0  # Net value of the contract
    cash_flows = defaultdict(float)  # Track daily cash flows

    # Process injections
    for date in injection_dates:
        date = pd.Timestamp(date)
        price = get_price(date)
        inject_volume = min(injection_rate, max_volume - storage_volume)
        if inject_volume > 0:
            storage_volume += inject_volume
            cash_flows[date] -= inject_volume * price  # Injection is a cost

    # Process withdrawals
    for date in withdrawal_dates:
        date = pd.Timestamp(date)
        price = get_price(date)
        withdraw_volume = min(withdrawal_rate, storage_volume)
        if withdraw_volume > 0:
            storage_volume -= withdraw_volume
            cash_flows[date] += withdraw_volume * price  # Withdrawal is a revenue

    # Apply storage costs
    for date in all_dates:
        daily_cost = storage_volume * storage_costs
        cash_flows[date] -= daily_cost

    # Calculate net value of the contract
    net_value = sum(cash_flows.values())

    return net_value

In [8]:
# Example usage:
injection_dates = ["2024-12-01", "2024-12-02"]
withdrawal_dates = ["2024-12-10", "2024-12-11"]

injection_rate = 10
withdrawal_rate = 8
max_volume = 50
storage_costs = 0.5
start_date = "2024-12-01"

net_value = price_contract(
    injection_dates,
    withdrawal_dates,
    df,
    injection_rate,
    withdrawal_rate,
    max_volume,
    storage_costs,
    start_date
)
print(f"Net value of the contract: {net_value}")


Net value of the contract: 48.81955805753344
